Импортируем все что может понадобится

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, roc_curve, RocCurveDisplay, auc, confusion_matrix, top_k_accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit, cross_validate
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import warnings
import datetime
import random
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from category_encoders.one_hot import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.cluster import DBSCAN
from catboost import CatBoostClassifier
from tqdm import tqdm

# Начинаем загрузку данных и добавляем некоторые новые признаки

Загружаем таблицу с данными пользователей

In [2]:
users=pd.read_csv("/kaggle/input/data-for-exp/users.xls")
users=users.drop(["Unnamed: 0"], axis=1)
users

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


Загружаем таблицу с данными действий пользователей, при просмотре постов, извлекаем признаки из даты и времени

In [3]:
feed=pd.read_csv("/kaggle/input/data-for-exp/feed.xls")
feed=feed.sort_values(by=["timestamp"])

#post["likes"]=0
#for x in tqdm(post["post_id"]):
    #luk=feed.loc[feed["post_id"]==x, "target"].sum()
    #post.loc[x, "likes"]=luk
#feed=feed.sort_values("timestamp")
feed=feed.drop(["Unnamed: 0"], axis=1)
feed["timestamp"]=pd.to_datetime(feed["timestamp"], format='%Y-%m-%d %H:%M:%S')


feed['week'] = feed["timestamp"].dt.isocalendar().week
feed['day_of_week'] = feed["timestamp"].dt.dayofweek
feed['hour'] = feed["timestamp"].dt.hour
feed['month'] = feed["timestamp"].dt.month

Загружаем таблицу с данными постов

In [4]:
posts=pd.read_csv("/kaggle/input/posts-new/123 (1).xls", sep=";")
posts=posts.drop(['vector_square', 'vector_sum', 'Unnamed: 0'], axis=1)

# Обработка признаков

Соединяем три таблицы в одну, убираем пропуски

In [5]:
merge_fp=feed.merge(posts, how="inner", left_on="post_id", right_on="post_id")
merge=merge_fp.merge(users, how="outer", left_on="user_id", right_on="user_id")
merge=merge.fillna(0)
merge.loc[merge["topic"]==0, "topic"]="No category"
merge=merge[merge["action"]!="like"]
userse=merge["user_id"]

merge=merge.drop(["action", "timestamp"], axis=1)
merge

,user_id,post_id,target,week,day_of_week,hour,month,0,1,2,...,text,topic,size text,gender,age,country,city,exp_group,os,source
0,84869,6826.0,0.0,39,4.0,6.0,10.0,-0.056640,0.138333,0.181871,...,"Wow, I havent seen a movie this bad since Fire...",movie,397.0,1,23,Russia,Gorodets,2,iOS,ads
1,84869,4272.0,0.0,39,6.0,11.0,10.0,-0.067361,0.156545,0.250135,...,"I enjoyed Ramin Bahranis Man Push Cart, and th...",movie,904.0,1,23,Russia,Gorodets,2,iOS,ads
2,84869,988.0,1.0,39,4.0,6.0,10.0,-0.116202,0.131734,0.272763,...,Blair looks to election campaign\n\nTony Blair...,politics,3512.0,1,23,Russia,Gorodets,2,iOS,ads
4,84869,462.0,0.0,48,1.0,21.0,11.0,-0.024221,0.138512,0.259009,...,Steady job growth continues in US\n\nThe US cr...,business,2182.0,1,23,Russia,Gorodets,2,iOS,ads
5,84869,1664.0,0.0,51,2.0,7.0,12.0,-0.071560,0.152003,0.331017,...,Benitez issues warning to Gerrard\n\nLiverpool...,sport,875.0,1,23,Russia,Gorodets,2,iOS,ads
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4653724,168548,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0,No category,0.0,0,36,Russia,Kaliningrad,4,Android,organic
4653725,168549,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0,No category,0.0,0,18,Russia,Tula,2,Android,organic
4653726,168550,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0,No category,0.0,1,41,Russia,Yekaterinburg,4,Android,organic
4653727,168551,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0,No category,0.0,0,38,Russia,Moscow,3,iOS,organic


Кодируем признаки

In [6]:
categorical=merge.loc[:, merge.dtypes==np.object].columns.tolist()
cat_ohe=["exp_group", "os", "source"]
for col in cat_ohe:
    df=pd.get_dummies(merge[col], drop_first=True, prefix=col)
    merge=pd.concat((merge, df), axis=1)
merge=merge.drop(cat_ohe, axis=1)
categorical.remove("os")
categorical.remove("source")
categorical.remove("text")
print(categorical)
for x in categorical:
    c=merge.groupby(x).agg({'target':'mean'})
    print(x)
    for z in tqdm(c.index):
        merge.loc[merge[x]==z, x]=c.loc[c.index==z].values
merge=merge.drop(["text"], axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


['topic', 'country', 'city']
topic


100%|██████████| 8/8 [00:03<00:00,  2.05it/s]


country


100%|██████████| 11/11 [00:05<00:00,  1.94it/s]


city


100%|██████████| 3915/3915 [29:47<00:00,  2.19it/s]


Загружаем и присоединяем таблицу с отношением просмотров и лайков для каждого пользователя, по каждой теме

In [ ]:
mer=pd.read_csv("/kaggle/input/full-rates/scaled_user_views_features_clean.csv", sep=";")
merge=merge.merge(mer, how="inner", left_on="user_id", right_on="user_id")

Проверяем какие колонки есть в таблице

In [8]:
for o in merge.columns:
    print(o)

user_id
post_id
target
week
day_of_week
hour
month
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
topic
size text
gender
age
country
city
exp_group_1
exp_group_2
exp_group_3
exp_group_4
os_iOS
source_organic
movie
covid
sport
politics
business
entertainment
tech


Разбиваем на таблицы постов и пользователей

In [16]:
u=merge.drop(['post_id', 'target', 'week', 'day_of_week', 'hour', 'month', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', 'topic', 'size text'], axis=1)
p=merge.drop(['gender', 'movie', 'covid', 'sport', 'politics', 'business', 'entertainment', 'tech', 'size text', 'user_id', 'target', 'week', 'day_of_week', 'hour', 'month', 'age', 'country', 'city', 'exp_group_1', 'exp_group_2', 'exp_group_3', 'exp_group_4', 'os_iOS', 'source_organic'], axis=1)

Убираем дубликаты из таблиц и сохраняем их, далее их будем отправлять в БД, чтобы потом во время работы приложения, быстро их скачать

In [17]:
u.drop_duplicates(subset=['user_id'], inplace=True)
p.drop_duplicates(subset=['post_id'], inplace=True)
u.to_csv("u-2.csv", sep=";", index=False)
p.to_csv("p-2.csv", sep=";", index=False)

делим общую таблицу на тест и трейн

In [13]:
mergeX_train, mergeX_test, mergeY_train, mergeY_test=train_test_split(merge.drop(["target", 'size text'], axis=1), merge["target"], test_size=0.2)

# Самая важная часть

Проверяем какие результаты получаются

In [14]:
catboost=CatBoostClassifier(scale_pos_weight=7.62, learning_rate=0.88, has_time=True, random_state=42, iterations=317, depth=6)
catboost.fit(merge.drop(["target", "size text"], axis=1), merge["target"])
tn, fp, fn, tp=confusion_matrix(mergeY_test, catboost.predict(mergeX_test)).ravel()
alll=tn+fp+fn+tp
print(F'Правильно определены - true negative {tn} - {tn/alll*100}%.')
print(F'Правильно определены - true positive {tp} - {tp/alll*100}%.')
print(F'Неправильно определены - false positive {fp} - {fp/alll*100}%.')
print(F'Неправильно определены - false negative {fn} - {fn/alll*100}%.')
print(F'Общая метрика составляет {(tn+tp)/alll*100}%')
print(F'Отношение tp к fp - {tp/fp}, общее число рекомендаций - {tp+fp}')

0:	learn: 0.6502365	total: 2.27s	remaining: 11m 55s
1:	learn: 0.6443502	total: 4.03s	remaining: 10m 34s
2:	learn: 0.6329086	total: 6.29s	remaining: 10m 57s
3:	learn: 0.6317521	total: 8.26s	remaining: 10m 46s
4:	learn: 0.6297316	total: 10s	remaining: 10m 25s
5:	learn: 0.6286101	total: 11.6s	remaining: 10m 3s
6:	learn: 0.6281641	total: 13.5s	remaining: 9m 57s
7:	learn: 0.6275954	total: 15.3s	remaining: 9m 50s
8:	learn: 0.6265763	total: 17.6s	remaining: 10m 3s
9:	learn: 0.6255975	total: 19.6s	remaining: 10m
10:	learn: 0.6250808	total: 21.4s	remaining: 9m 54s
11:	learn: 0.6245433	total: 23.3s	remaining: 9m 51s
12:	learn: 0.6239669	total: 25.1s	remaining: 9m 47s
13:	learn: 0.6238028	total: 26.6s	remaining: 9m 34s
14:	learn: 0.6234623	total: 28.1s	remaining: 9m 26s
15:	learn: 0.6229337	total: 30s	remaining: 9m 23s
16:	learn: 0.6225215	total: 31.9s	remaining: 9m 23s
17:	learn: 0.6218646	total: 34s	remaining: 9m 25s
18:	learn: 0.6214908	total: 35.8s	remaining: 9m 21s
19:	learn: 0.6209911	total

# Побочные ячейки

Рисуем график

In [ ]:
fpr, tpr, threshold, = roc_curve(mergeY_test, catboost.predict_proba(mergeX_test)[:, 1])
RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

Считаем AUC

In [ ]:
auc(fpr, tpr)

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay, precision_recall_curve
precision, recall, thresholds = precision_recall_curve(mergeY_test, catboost.predict_proba(mergeX_test)[:, 1])

# И нарисуем PR-кривую



PrecisionRecallDisplay(precision=precision, recall=recall).plot()

# Посчитаем PR-AUC
auc(recall, precision)

In [15]:
catboost.save_model('catboost',
                           format="cbm")